In [4]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import re

In [6]:
file_path = 'C://Users/dadel/Downloads/202409_Kazakhstan companies selection v3.xlsx'  # Update the path if different
df = pd.read_excel(file_path)

In [8]:
df = pd.read_excel(file_path, skiprows=1)
df

,Unnamed: 0,Company name,Address,Management,Phone,Email,Line of business/sector,ISIC Code,Average number of employees
0,1,"TOO ""PERVY PIVZAVOD""","ALMATINSKAYA OBLAST, ILISKI RAION, ASHCHIBULAK...",AIDARBEKOV VLADIMIR ORALKHANOVICH,+7(727)3210000,almatybeer@mail.kz,MALT BEVERAGES,2082,251 - 500
1,2,"TOO ""ENGRAVE""","G. ALMATY, MEDEUSKI RAION, ULITSA KHADZHI MUKA...",GORYACHKO ROMAN ALEKSEEVICH,+7(777)6666699,info@engrave.kz,MISCELLANEOUS METALWORK,3449,NaN
2,3,"TOO ""ARMAN""","AKMOLINSKAYA OBLAST, ZHAKSYNSKI RAION, KALININ...",RAEVA SAULE UAKHITOVNA,+7(716)3522611,raeva-su@mail.ru,"FIELD CROPS, EXCEPT CASH GRAIN",139,51 - 100
3,4,"TOO ""BESTEREK-2014""","AKMOLINSKAYA OBLAST, TSELINOGRADSKI RAION, TAL...",TOIGULIEV BAKTYBEK MAKSUTOVICH,+7(700)5151547,zhandos.1980@mail.ru,"GENERAL FARMS, PRIMARILY CROP",191,NaN
4,5,"TOO ""BI TEMIR""","G. SHYMKENT, AL-FARABISKI RAION, ULITSA ERIMBE...",TASTANBEK OLZHAS MAKHANBETELIULY,+7(702)3577077,too777@inbox.ru,BLAST FURNACES AND STEEL MILLS,3312,NaN
...,...,...,...,...,...,...,...,...,...
317,318,"AO ""ALYUMINI KAZAKHSTANA""","PAVLODARSKAYA OBLAST, PAVLODAR G. A., G. PAVLO...",KRASNOYARSKI VLADIMIR,+7(7182)370111\n+7(7182)370808\n+7(7182)370293...,aok.paz@erg.kz,PRIMARY ALUMINUM,3334,> 1000
318,319,"TOO ""RAZDOLE""","ZAPADNO-KAZAKHSTANSKAYA OBLAST, RAION BEITEREK...",TALIEV MUKHIT ZHULDAGALIEVICH,+7(711)3120265,razdolnenskiy_okrug@mail.ru,"FIELD CROPS, EXCEPT CASH GRAIN",139,51 - 100
319,320,"TOO ""VISHNEVSKOE""","SEVERO-KAZAKHSTANSKAYA OBLAST, TAIYNSHINSKI RA...",SHCHAVINSKI VLADIMIR PAVLOVICH,+7(715)3672535,disp111@mail.ru,"FIELD CROPS, EXCEPT CASH GRAIN",139,201 - 250
320,321,"TOO ""SARDAR EXPORT""","OBLAST ZHETISU, TALDYKORGAN G. A., G. TALDYKOR...",ISLAMKHAN MIRAS MEDIULY,+7(701)2414580\n+7(707)9166492,sardar.2030ltd@mail.ru,"BREAD, CAKE, AND RELATED PRODUCTS",2051,21 - 30


## Create mapping with GTAP

In [17]:
mapping_df

,HS6,GTAP,GTAP description,HS6 description,HS2
0,101.21,ctl,"Cattle, sheep, goats, horses",Pure-bred breeding horses,10
1,101.29,ctl,"Cattle, sheep, goats, horses",Live horses (excl. pure-bred for breeding),10
2,101.30,ctl,"Cattle, sheep, goats, horses",Live asses,10
3,101.90,ctl,"Cattle, sheep, goats, horses",Live mules and hinnies,10
4,102.21,ctl,"Cattle, sheep, goats, horses",Pure-bred cattle for breeding,10
...,...,...,...,...,...
5382,9702.00,omf,Manufactures nec,"Original engravings, prints and lithographs",97
5383,9703.00,omf,Manufactures nec,"Original sculptures and statuary, in any material",97
5384,9704.00,ome,Machinery and equipment nec,"Postage or revenue stamps, stamp-postmarks, fi...",97
5385,9705.00,omf,Manufactures nec,Collections and collector's pieces of zoologic...,97


In [18]:
mapping_data_path = 'C://Users/dadel/Downloads/10977.xlsx'
mapping_df = pd.read_excel(mapping_data_path)
mapping_df['HS2'] = mapping_df['HS6'].apply(lambda x: str(x)[:2] if len(str(x)) >= 7 else '0' + str(x)[:1])
gtap_mapping = mapping_df.set_index('HS2')['GTAP'].to_dict()
df['HS2'] = df['ISIC Code'].apply(lambda x: str(x)[:2] if len(str(x)) >= 4 else '0' + str(x)[:1])
df['GTAP'] = df['HS2'].map(gtap_mapping)
df.head()  

,Unnamed: 0,Company name,Address,Management,Phone,Email,Line of business/sector,ISIC Code,Average number of employees,HS2,GTAP
0,1,"TOO ""PERVY PIVZAVOD""","ALMATINSKAYA OBLAST, ILISKI RAION, ASHCHIBULAK...",AIDARBEKOV VLADIMIR ORALKHANOVICH,+7(727)3210000,almatybeer@mail.kz,MALT BEVERAGES,2082,251 - 500,20,ofd
1,2,"TOO ""ENGRAVE""","G. ALMATY, MEDEUSKI RAION, ULITSA KHADZHI MUKA...",GORYACHKO ROMAN ALEKSEEVICH,+7(777)6666699,info@engrave.kz,MISCELLANEOUS METALWORK,3449,NaN,34,crp
2,3,"TOO ""ARMAN""","AKMOLINSKAYA OBLAST, ZHAKSYNSKI RAION, KALININ...",RAEVA SAULE UAKHITOVNA,+7(716)3522611,raeva-su@mail.ru,"FIELD CROPS, EXCEPT CASH GRAIN",139,51 - 100,01,ofd
3,4,"TOO ""BESTEREK-2014""","AKMOLINSKAYA OBLAST, TSELINOGRADSKI RAION, TAL...",TOIGULIEV BAKTYBEK MAKSUTOVICH,+7(700)5151547,zhandos.1980@mail.ru,"GENERAL FARMS, PRIMARILY CROP",191,NaN,01,ofd
4,5,"TOO ""BI TEMIR""","G. SHYMKENT, AL-FARABISKI RAION, ULITSA ERIMBE...",TASTANBEK OLZHAS MAKHANBETELIULY,+7(702)3577077,too777@inbox.ru,BLAST FURNACES AND STEEL MILLS,3312,NaN,33,crp


## Deal with Size

In [19]:
def process_employee_range(value):
    if pd.isna(value):
        return np.nan  # Keep NaN for now, we will fill later
    elif ">" in value:
        return 1000  # For entries with "greater than 1000", set it to 1000
    elif "-" in value:
        # Extract the middle of the range
        low, high = value.split("-")
        return (int(low.strip()) + int(high.strip())) / 2
    else:
        return np.nan  # Handle any unexpected format as NaN

In [20]:
df['Average_employees_estimated'] = df['Average number of employees'].apply(process_employee_range)

# Calculate the mean of non-NaN values in the new column
mean_value = df['Average_employees_estimated'].mean()

# Fill NaN values with the calculated mean
df['Average_employees_estimated'].fillna(mean_value, inplace=True)

# Display the updated DataFrame
df.head()

,Unnamed: 0,Company name,Address,Management,Phone,Email,Line of business/sector,ISIC Code,Average number of employees,HS2,GTAP,Average_employees_estimated
0,1,"TOO ""PERVY PIVZAVOD""","ALMATINSKAYA OBLAST, ILISKI RAION, ASHCHIBULAK...",AIDARBEKOV VLADIMIR ORALKHANOVICH,+7(727)3210000,almatybeer@mail.kz,MALT BEVERAGES,2082,251 - 500,20,ofd,375.500000
1,2,"TOO ""ENGRAVE""","G. ALMATY, MEDEUSKI RAION, ULITSA KHADZHI MUKA...",GORYACHKO ROMAN ALEKSEEVICH,+7(777)6666699,info@engrave.kz,MISCELLANEOUS METALWORK,3449,NaN,34,crp,136.323256
2,3,"TOO ""ARMAN""","AKMOLINSKAYA OBLAST, ZHAKSYNSKI RAION, KALININ...",RAEVA SAULE UAKHITOVNA,+7(716)3522611,raeva-su@mail.ru,"FIELD CROPS, EXCEPT CASH GRAIN",139,51 - 100,01,ofd,75.500000
3,4,"TOO ""BESTEREK-2014""","AKMOLINSKAYA OBLAST, TSELINOGRADSKI RAION, TAL...",TOIGULIEV BAKTYBEK MAKSUTOVICH,+7(700)5151547,zhandos.1980@mail.ru,"GENERAL FARMS, PRIMARILY CROP",191,NaN,01,ofd,136.323256
4,5,"TOO ""BI TEMIR""","G. SHYMKENT, AL-FARABISKI RAION, ULITSA ERIMBE...",TASTANBEK OLZHAS MAKHANBETELIULY,+7(702)3577077,too777@inbox.ru,BLAST FURNACES AND STEEL MILLS,3312,NaN,33,crp,136.323256


## Location